In [1]:
#importing libraries

import datetime
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import xarray as xr
import cftime
import cartopy
from dateutil.relativedelta import relativedelta
import matplotlib as mpl
from matplotlib import colors
from scipy.interpolate import griddata

import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from matplotlib.patches import Path, PathPatch
import cartopy.feature as cfeature
import cartopy.crs as ccrs

In [2]:
full_narr = xr.open_mfdataset("/Users/dessyb/Library/CloudStorage/Box-Box/Patrick_MS/data/acpcp_data/daily/subset/acpcp.*.nc")

/opt/miniconda3/envs/MPO624/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'acpcp' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/miniconda3/envs/MPO624/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'acpcp' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/miniconda3/envs/MPO624/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'acpcp' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/miniconda3/envs/MPO624/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'acpcp' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/opt/miniconda3/envs/MPO624/lib/python3.

In [3]:
month_slice = full_narr.sel(time=full_narr.time.dt.month.isin([1,2,3]))

In [4]:
J_E1 = xr.open_mfdataset("/Volumes/NMME_Daily/CCSM4.DailyT2mPrec.JanIC.E1.*-01-*.nc")

In [5]:
lon = np.array(J_E1.lon.values)

In [7]:
lat = np.array(J_E1.lat.values)

In [8]:
from pyresample import kd_tree, geometry

ModuleNotFoundError: No module named 'pyresample'

In [4]:
n_acpcp = np.array(month_slice.acpcp)

In [5]:
f_narr = n_acpcp.flatten()

In [6]:
narr_times = (24*np.arange(0,2888,1))

In [7]:
sub_lon = np.array(month_slice.lon)
sub_lat = np.array(month_slice.lat)

array([[-145.5     , -145.315   , -145.1295  , ...,  -68.68951 ,
         -68.50452 ,  -68.32005 ],
       [-145.6047  , -145.4194  , -145.2336  , ...,  -68.58509 ,
         -68.39979 ,  -68.215   ],
       [-145.71    , -145.5244  , -145.3383  , ...,  -68.48015 ,
         -68.29454 ,  -68.10945 ],
       ...,
       [ 149.476   ,  149.555   ,  149.6349  , ...,   -3.556949,
          -3.477873,   -3.399673],
       [ 149.0593  ,  149.1361  ,  149.2138  , ...,   -3.137999,
          -3.061175,   -2.985206],
       [ 148.6418  ,  148.7163  ,  148.7917  , ...,   -2.718174,
          -2.643617,   -2.569891]], dtype=float32)

In [8]:
sub_lon2 = sub_lon.flatten()
sub_lat2 = sub_lat.flatten()

In [9]:
f_lon = list(sub_lon2) * 2888
f_lat = list(sub_lat2) * 2888

In [10]:
f_time = np.repeat(narr_times, 277*349)

In [49]:
df1 = pd.DataFrame({'lat':np.array(f_lat),'lon':np.array(f_lon),'hours':f_time,'acpcp':f_narr})

In [12]:
J_E1 = xr.open_mfdataset("/Volumes/NMME_Daily/CCSM4.DailyT2mPrec.JanIC.E1.*-01-*.nc")
F_E1 = xr.open_mfdataset("/Volumes/NMME_Daily/CCSM4.DailyT2mPrec.FebIC.E1.*-02-*.nc")
M_E1 = xr.open_mfdataset("/Volumes/NMME_Daily/CCSM4.DailyT2mPrec.MarIC.E1.*-03-*.nc")

In [13]:
J_E1 = J_E1.sel(time=J_E1.time.dt.year.isin(np.arange(1989,2021,1)))
F_E1 = F_E1.sel(time=F_E1.time.dt.year.isin(np.arange(1989,2021,1)))
M_E1 = M_E1.sel(time=M_E1.time.dt.year.isin(np.arange(1989,2021,1)))

In [15]:
J_E1 = J_E1.sel(lat=slice(0,90))
F_E1 = F_E1.sel(lat=slice(0,90))
M_E1 = M_E1.sel(lat=slice(0,90))

In [17]:
E1 = xr.merge([J_E1, F_E1, M_E1], join="outer")

In [80]:
E1

<xarray.Dataset>
Dimensions:  (time: 2880, lat: 164, lon: 288)
Coordinates:
  * time     (time) object 1989-01-01 00:00:00 ... 2020-03-31 00:00:00
  * lat      (lat) float64 0.4712 0.4712 1.414 1.414 ... 88.12 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    PRECT    (time, lat, lon) float32 dask.array<chunksize=(32, 164, 288), meta=np.ndarray>
    TREFHT   (time, lat, lon) float32 dask.array<chunksize=(32, 164, 288), meta=np.ndarray>

In [46]:
time = []

for i in range(len(E1.time)):
    time.append(24*((E1.time.values[i]-E1.time.values[0]).days))

In [57]:
sub_elon = np.array(E1.lon)-180
sub_elat = np.array(E1.lat)

In [62]:
e_lon = list(sub_elon) * len(time) * len(sub_elat)

In [65]:
e_lat_interim = np.repeat(sub_elat, len(sub_elon))

In [66]:
e_lat = list(e_lat_interim) * len(time)

In [69]:
e_time = np.repeat(time, len(sub_elon) * len(sub_elat))

In [71]:
df2 = pd.DataFrame({'lat':np.array(e_lat),'lon':np.array(e_lon),'hours':e_time})

In [77]:
df1_round = df1.round(4)
df2_round = df2.round(4)

In [85]:
df_3 = df1_round.head(500000)
df_4 = df1_round.head(500000)

In [ ]:
# inter_acpcp = griddata(df1_round.loc[:,['lat','lon','hours']].values,df1_round.iloc[:,3].values,df2_round.iloc[:,[0,1,2]].values)


inter_acpcp = griddata(df_3.loc[:,['lat','lon','hours']].values,df_3.iloc[:,3].values,df_4.iloc[:,[0,1,2]].values)

